In [9]:
import random as rd
import datetime as dt
class dataClass:
    start = ''
    end = ''
    form = '%Y-%m-%d %H:%M:%S'
    out = None
    error = None
    siramlog = None
    siramlogh = False
    last = []
    lasth = -1
    n = 5
    hari = None
    logharian = None
    namafile = None
    disiram = 0
    sudahdisiram = 0
    getarringan = ['100','010','001']
    getaracak = []
    getarberat = ['110','011','101','111']
    getaracakb = []
    humdur = []
    humdurc = 0
    humdurs = False
    btsiram = 8
    def __init__(self,n,file='Data.csv',
                 start='2013-01-01 00:00:00',
                 end='2014-01-01 00:00:00'):
        self.start=start
        self.end=end
        self.n=n
        for x in range(n):
            self.last.append(0)
        self.getaracak.extend(self.getarringan)
        for x in range(30):
            self.getaracak.append('000')
        self.getaracakb.extend(self.getarberat)
        for x in range(30):
            self.getaracakb.append('000')
        self.out = open(file,'w')
        filename = file.split('.')
        self.error = open(filename[0]+'-error.'+filename[1],'w')
        self.siramlog = open(filename[0]+'-siram.'+filename[1],'w')
        self.namafile = filename[0]
        self.out.write('date,humidity,temp,vibra,piezoVibra\n')
        self.error.write('date\n')
        self.siramlog.write('date,humidity,duration\n')
        self.randRow()
        self.siramlog.write(str(self.sudahdisiram)+'\n')
        print(self.humdur)
        print(self.sudahdisiram)
    def __del__(self):
        self.out.close()
        self.error.close()
        self.siramlog.close()
    def randRow(self):
        d = dt.datetime.strptime(self.start,self.form)
        e = dt.datetime.strptime(self.end,self.form)
        while d<e:
            d += dt.timedelta(seconds=1)
            if d.day != self.hari:
                if self.logharian != None:
                    self.logharian.close()
                self.hari = d.day
                self.logharian = open(self.namafile+'-'+str(d.month)+'-'+str(d.day)+'.csv','w')
            self.lasth = d.hour
            rw = self.randCol(d)
            errrate = rd.randint(1,100)
            if(errrate>95):
                self.error.write(rw[0]+"\n")
            else:
                rwstr = ",".join(rw)
                self.out.write(rwstr+"\n")
                self.logharian.write(rwstr+"\n")
    def randCol(self,d):
        out = []
        out.insert(0,d.strftime(self.form))
        col = self.colmaker(d.hour)
        i = 0
        for x in col:
            cl = self.resexcept(x,i)
            if i == 0 and self.siramlogh == True:
                self.humdur.append(self.humdurc)
                self.siramlog.write(str(d.strftime(self.form))+','+str(self.last[0])+','+str(self.humdurc)+'\n')
                self.humdurs = False
                
                self.siramlog.write(str(d.strftime(self.form))+','+str(cl)+','+str(self.humdurc)+'\n')
                self.siramlogh = False
                self.humdurc = 0
                self.humdurs = True
            elif i == 0 and self.humdurs == True:
                self.humdurc += 1
                if cl<=1:
                    self.humdur.append(self.humdurc)
                    self.siramlog.write(str(d.strftime(self.form))+','+str(self.last[0])+','+str(self.humdurc)+'\n')
                    self.humdurs = False
            out.append(str(cl))
            i+=1
        self.last[0] = float(out[1])
        self.last[1] = float(out[2])
        self.last[2] = out[3]
        return out
    def resexcept(self,x,i):
        if i == 0 and self.last[0]<0:
            cl = 0
        elif i in [2,3]:
            if self.disiram == 1 and self.sudahdisiram<self.btsiram:
                if i==2:
                    cl=self.getarringan[rd.randint(0,len(self.getarringan)-1)]
                else:
                    cl = rd.uniform(0,2)
            elif self.disiram == 1 and self.sudahdisiram>self.btsiram:
                if i==2:
                    cl=self.getarberat[rd.randint(0,len(self.getarberat)-1)]
                else:
                    cl = rd.uniform(2,4)
            elif self.disiram == 0 and self.sudahdisiram>self.btsiram:
                if i==2:
                    cl=self.getaracakb[rd.randint(0,len(self.getaracakb)-1)]
                else:
                    cl = rd.uniform(0,0.2)
            else:
                if i==2:
                    cl=self.getaracak[rd.randint(0,len(self.getaracak)-1)]
                else:
                    cl=0
            if i==2:
                cl = '\"'+cl+'\"'
            else:
                cl = round(cl,3)
        elif i in [0,1]:
            cl = rd.uniform(x[0],x[1])
            if i == 1 and cl>78:
                cl = rd.uniform(75,78)
            elif i == 1 and cl<18:
                cl = rd.uniform(16,18)
            elif i == 0 and cl>98:
                cl = rd.uniform(98,99)
            elif i == 0 and cl<10:
                cl = rd.uniform(10,11)    
            cl = round(cl,2)
        return cl
    def colmaker(self,h):
        out = [10,18,'000',0.0]
        humin = 0.008
        tempmin = 0.006
        if self.sudahdisiram>self.btsiram:
            add = ((self.sudahdisiram-self.btsiram)/10000)*2
            humin -= add
            tempmin += add
            
        if h in [8,16]:
            if self.disiram == 0 and (rd.randint(1,10)>5):
                out[0] = [98,100]
                self.disiram = 1
                self.sudahdisiram += 1
                self.siramlogh = True
            else:
                out[0] = [self.last[0]-humin,self.last[0]]
                
        elif h in [11,12,13,14]:
            out[0] = [self.last[0]-(humin*2),self.last[0]]
            self.disiram = 0
        elif h in [0,1,2,3,4,5]:
            out[0] = [self.last[0]-humin,self.last[0]+(humin/2)]
            self.disiram = 0
        else:
            out[0] = [self.last[0]-humin,self.last[0]]
            self.disiram = 0
        #other cols
        
        if h in [11,12,13]:
            if self.last[0]>40 and self.last[0]<=100:
                out[1] = [self.last[1],self.last[1]+tempmin]
            else:
                out[1] = [self.last[1],self.last[1]+tempmin*2]
        elif h in [7,8,9]:
            out[1] = [self.last[1],self.last[1]+tempmin]
        elif h in [16,17]:
            out[1] = [self.last[1]-tempmin,self.last[1]]
        elif h in [1,2,3]:
            out[1] = [self.last[1]-tempmin*2,self.last[1]]
        else:
            out[1] = [self.last[1]-tempmin,self.last[1]+tempmin]
        return out

test = dataClass(4,'DataTanah.csv',
        '2020-04-01 08:00:00',
        '2020-04-02 09:00:00')
del test

In [10]:
test = dataClass(4,'databesar2/DataTanahBesar.csv',
        '2020-04-12 08:12:32',
        '2020-04-20 17:03:21')
del test

[0, 28047, 57601, 28797, 57598, 28799, 57600, 28798, 57599, 28802, 57596, 28800, 57598, 28799, 57599, 28801, 57597, 28801]
18


In [8]:
print(0.006*2)
print(0.006/2)
print(0.008-(((9-8)/10000)*2))
print(((9-8)/10000)*2)

0.012
0.003
0.0078000000000000005
0.0002
